In [48]:
import numpy as np
import numpy
import scipy
import qiskit
from qiskit import transpile
import scipy.linalg as sl
import numpy.linalg as nl
from scipy.linalg import cossin
from qiskit.quantum_info import Operator, Statevector
from qiskit import QuantumCircuit
from qiskit.circuit.library import UGate,UCPauliRotGate
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Operator
import sys
sys.path.append('./src')
##
import utils_synth
import oracle_synth
import src.lcu as lcu
import lchs
import importlib
importlib.reload(lcu)
importlib.reload(lchs)
importlib.reload(utils_synth)
importlib.reload(oracle_synth)
##
from utils_synth import *
from oracle_synth import *
from lchs import *
from src.lcu import *

rng = numpy.random.default_rng(4290962056973923694427)
np.set_printoptions(precision=8, suppress=True)

/Users/zhen002/GitHub/qla_lib/src/lcu.py:63: SyntaxWarning: invalid escape sequence '\s'
  '''


In [49]:
test_coefs = numpy.array([1,2,3,4])
test_coefs_normed = test_coefs/numpy.linalg.norm(test_coefs, ord=1)
test_unitaries = [numpy.array([[1,0],[0,1]]), numpy.array([[0,1],[1,0]]), numpy.array([[1,0],[0,-1]]), numpy.array([[0,1j],[1j,0]])]
qiskit_api = True

# test_coefs = numpy.array([1,2])
# test_coefs_normed = test_coefs/numpy.linalg.norm(test_coefs, ord=1)
# test_unitaries = [numpy.array([[1,0],[0,1]]), numpy.array([[0,1],[1,0]])]

## -------------------
if len(test_coefs) != len(test_unitaries):
    raise ValueError("The number of coefficients and unitaries should be the same, but we have", len(test_coefs), "coefficients and", len(test_unitaries), "unitaries")
##
correct_answer = numpy.zeros(test_unitaries[0].shape, dtype=complex)
for i in range(len(test_coefs_normed)):
    correct_answer += test_coefs_normed[i]*test_unitaries[i]

## -------------------
PREP = prep_oracle(test_coefs, qiskit_api=qiskit_api)
SELE = select_oracle(test_unitaries, qiskit_api=qiskit_api)
LCU = lcu_generator(test_coefs, test_unitaries, verbose=1, qiskit_api=qiskit_api)
circ_op = qiskit.quantum_info.Operator(LCU.reverse_bits()).data
lcu_sol = circ_op[:test_unitaries[0].shape[0],:test_unitaries[0].shape[1]]
##
print("Correct answer:", correct_answer)
print("\nLCU Implementation:", lcu_sol)
print("\nError:", numpy.linalg.norm(correct_answer - lcu_sol, ord=2))

  LCU-Oracle: num_qubits_control= 2 num_qubits_op= 1
Correct answer: [[ 0.4+0.j   0.2+0.4j]
 [ 0.2+0.4j -0.2+0.j ]]

LCU Implementation: [[ 0.4+0.j   0.2+0.4j]
 [ 0.2+0.4j -0.2-0.j ]]

Error: 4.94988483328197e-15


In [47]:
PREP

Instruction(name='PREP', num_qubits=2, num_clbits=0, params=[(0.31622776601683794+0j), (0.4472135954999579+0j), (0.5477225575051661+0j), (0.6324555320336759+0j)])

In [ ]:
scipy.stats.unitary_group.random_state

In [31]:
from qiskit.transpiler.passes import RemoveResetInZeroState

qiscirc = qiskit.QuantumCircuit(2)
qiscirc.initialize(test_coefs_normed)
qiscirc = qiscirc.reverse_bits()
qiskit.transpiler.passes
qis_oracle = qiscirc.decompose(reps=6).to_gate()
qis_oracle.name = 'PREP'

QiskitError: 'One or more instructions cannot be converted to a gate. "reset" is not a gate instruction'

In [39]:
from qiskit.circuit.library import StatePreparation
StatePreparation(test_coefs_normed).name = "PREP"

In [41]:
import qiskit.quantum_info


qiskit.quantum_info.Operator(StatePreparation(test_coefs_normed)).data

array([[ 0.31622777-0.j        ,  0.        +0.4472136j ,
        -0.48304589+0.j        , -0.        -0.68313005j],
       [ 0.4472136 -0.j        , -0.        -0.31622777j,
        -0.68313005+0.j        ,  0.        +0.48304589j],
       [ 0.54772256+0.j        , -0.        -0.63245553j,
         0.35856858+0.j        , -0.        -0.41403934j],
       [ 0.63245553+0.j        , -0.        +0.54772256j,
         0.41403934+0.j        , -0.        +0.35856858j]])

In [58]:
tmp = list(dict({'1':1, '2':2}).values())
tmp

[1, 2]